### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using aql.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [2]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Runing aql in iJava
We can run aql commands in a file. No output will be displayed. 
In _trucate_test.aql_, we have a single AQL command: TRUNCATE test



In [3]:
%sh aql -c "truncate test" -h "3.235.121.162"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [4]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.0.0</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [12]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.Operation;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpWriteFlags;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.exp.ListExp;

import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

AerospikeClient client = new AerospikeClient("3.235.121.162", 3000);
Key key = new Key("test", "testset", "key1");
WritePolicy wPolicy = new WritePolicy();
Bin bin1 = new Bin("iBin1", Value.get(1));
client.delete(null, key);
System.out.println("Record not exists initially: "+ client.get(null, key)); //check record got inserted

Expression recExists = Exp.build(Exp.keyExists()) ; 

wPolicy.filterExp = recExists;                          

client.put(wPolicy, key, bin1);  //First update
System.out.println("Record - add bin1: " + client.get(null, key));
Bin bin2 = new Bin("iBin2", Value.get(2)); //int bin
client.put(wPolicy, key, bin2);  //Second update
System.out.println("Record - add bin2, fails to update: " + client.get(null, key));


Record not exists initially: null
Record - add bin1: (gen:1),(exp:418190316),(bins:(iBin1:1))
Record - add bin2, fails to update: (gen:1),(exp:418190316),(bins:(iBin1:1))
